In [0]:
!pip install Faker

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached Faker-33.3.1-py3-none-any.whl (1.9 MB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
#IMPORT LIBRARIES
from faker import Faker #generate fake data
import random #generate pseudo-random numbers
from pyspark.dbutils import DBUtils #work with files and object storage efficiently
from faker.providers import internet # generate fake data
from datetime import datetime #manipulating dates and times
import pandas as pd #data manipulation
import os #os interaction

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-1379593343009408>, line 2
      1 #IMPORT LIBRARIES
----> 2 from faker import Faker #generate fake data
      3 import random #generate pseudo-random numbers
      4 from pyspark.dbutils import DBUtils #work with files and object storage efficiently

ModuleNotFoundError: No module named 'faker'

In [0]:
dbutils.fs.help("cp")

/** * Copies a file or directory, possibly across FileSystems.. * * Example: cp("/mnt/my-folder/a", "s3n://bucket/b") * * @param from FileSystem URI of the source file or directory * @param to FileSystem URI of the destination file or directory * @param recurse if true, all files and directories will be recursively copied * @return true if all files were successfully copied */ cp(from: java.lang.String, to: java.lang.String, recurse: boolean = false): boolean

In [0]:
def replace_rename_parquet(location, new_location):
  """
  Renames and moves parquet file to a new location, then removes the original directory.

  Parameters:
  location (str): the present file location.
  new_location (str): the file's new location.

  Returns:
  A print statement of the file's new location.
  """
  from datetime import datetime

  for file_info in dbutils.fs.ls(location):
    #return of dbutils.fs.ls: [FileInfo(path='dbfs:/mnt/data/original_folder/data_2024-11-15.parquet', name='data_2024-11-15.parquet', size=1024)]
    
    file_path = file_info.path #getting the path

    if file_path[-8:] == '.parquet':
      #If file is a .parquet, move the file to the new location and remove the previous.
      dbutils.fs.cp(file_path, new_location)
      dbutils.fs.rm(location, True)

  return print(f'File generated at: {new_location}')


In [0]:
dbutils.fs.help("rm")

/** * Removes a file or directory. * * Example: rm("/mnt/my-folder/", true) * * @param dir FileSystem URI for a single file or a directory * @param recurse if true, all files and directories will be recursively deleted * @return true if the file or directory was present and is now deleted */ rm(dir: java.lang.String, recurse: boolean = false): boolean

In [0]:
#generating data

fake = Faker()
fake.add_provider(internet)

def gen_data(connections_quantity, device, status):
  '''
  Creates fake data of devices connections.

  Parameters:
  connections_quantity (int): the quantity of devices connections on the network.
  device (str): the device that it'll be connecting.
  status (str): the connections's status.
  '''
  
  fake_records = []
  speed_connection = [1, 5, 10, 15, 25, 35, 50, 100, 200, 400, 500, 1000]

  #generating fake data list
  for i in range(connections_quantity):
    new_register = {
      'Name': fake.name(),
      'Address': fake.address(),
      'IP': fake.ipv4_private(),
      'Connection_Time': datetime.now(),
      'Device': device,
      'Speed_Connection': random.choice(speed_connection),
      'Connection_Status': status
    }
    fake_records.append(new_register)

  # 'from_records' method creates a dataframe from tuples, dicts
  df = pd.DataFrame.from_records(fake_records)  
  # converting pandas df to spark df
  spark_df = spark.createDataFrame(df)


  #saving the data in the landing zone:
  try:
    #path of the generated file
    local = f"dbfs:/FileStore/landing_zone/{device}/{status}/{status}_{datetime.now()}"
    #write file
    spark_df.coalesce(1).write.format("parquet").mode("overwrite").save(local)
    new_local = f"dbfs:/FileStore/landing_zone/{device}/{status}/{device}/{device}_{datetime.now()}.parquet"

    replace_rename_parquet(local, new_local)

    return print(f'Sucessfully generated: {connections_quantity} connections of {device} | Status: {status} | Path: {new_local}')
  
  except:
    return print("Error at gen_data().")

In [0]:
def ingestion_gen_data(argument):
  '''
  Generates the exact data we want for the ingestion.
  
  Parameters:
  argument (str): key of the data we want to ingest.

  Example:
  ingestion_gen_data('Computer_Online')
  '''

  switch_case = {
    'Computer_Online': lambda: gen_data(40, 'Computer', 'Online'),
    'Mobile_Online': lambda: gen_data(80, 'Mobile', 'Online'),
    'Computer_Offline': lambda: gen_data(70, 'Computer', 'Offline'),
    'Mobile_Offline': lambda: gen_data(60, 'Mobile', 'Offline')
  }
  
  #executes an item of switch_case
  switch_case.get(argument, lambda: print('Not found!'))()

In [0]:
def delete_directory(dbutils, directory_path):
    if dbutils.fs.mkdirs(f"{directory_path}"):
        files = dbutils.fs.ls(f"{directory_path}")
        if files:
            for file in files:
                if file.isDir():
                    delete_directory(dbutils, file.path)
                    print(f"Files in the directory {file} excluded with success.")
                else:
                    dbutils.fs.rm(file.path)
                    print(f"Files in the directory {file} excluded with success ")
                    
        dbutils.fs.rm(directory_path)

        return print(f"Files in the directory {directory_path} excluded with success")
    else:
        return print(f"The directory {directory_path} doesn`t exists.")